# 🔌 Agents with Foundry MCP Server

Welcome to our **Foundry MCP Server** tutorial! In this notebook, we'll demonstrate how to:

1. **Understand** the Foundry MCP Server and its capabilities
2. **Configure** an MCP tool connection in Microsoft Foundry
3. **Create an Agent** with MCP tools to access Foundry capabilities
4. **Handle MCP approval requests** for secure tool access

### 🔌 What is Foundry MCP Server?
> [Foundry MCP Server](https://learn.microsoft.com/azure/ai-foundry/mcp/get-started) is a **cloud-hosted MCP service** at `https://mcp.ai.azure.com` that provides:
> - **Model Discovery**: Explore and compare models from the Foundry catalog
> - **Model Deployment**: Check quotas and deploy models
> - **Agent Management**: Create and manage AI agents
> - **Evaluation**: Run evaluations and quality assessments
> - **Knowledge Integration**: Work with Azure AI Search indexes

### 🌟 Why Foundry MCP Server?
| Feature | Benefit |
|---------|---------|
| **Cloud-hosted** | No local server setup required |
| **Entra ID Authentication** | Secure OAuth 2.0 with user-specific permissions |
| **Curated Tools** | Pre-built tools for common Foundry operations |
| **Scalable** | Built with cloud-scale reliability |

## Prerequisites
- Microsoft Foundry project with appropriate permissions
- **Foundry MCP Server connection** configured in your project (see setup below)
- A `.env` file in the parent directory containing:
  ```bash
  AI_FOUNDRY_PROJECT_ENDPOINT=<your-ai-foundry-project-endpoint>
  AZURE_AI_MODEL_DEPLOYMENT_NAME=<supported-model>
  TENANT_ID=<your-azure-tenant-id>
  FOUNDRY_MCP_CONNECTION_ID=<connection-id-from-foundry-portal>  # See setup below
  ```

## ⚙️ Setting up Foundry MCP Server Connection

**Important**: Before using MCP tools in code, you must create a connection in the Foundry portal:

### Step 1: Navigate to Foundry Tools
1. Go to [Microsoft Foundry portal](https://ai.azure.com)
2. Select your project
3. Navigate to **Build** → **Tools**

### Step 2: Add Foundry MCP Server
1. Click **Add tool** → **Catalog**
2. Search for "Foundry MCP Server (preview)"
3. Select it and click **Create**
4. The connection will be created with OAuth authentication

### Step 3: Get Connection ID
1. After creating the connection, click on it to view details
2. Copy the **Project connection ID** from the details page
3. Add it to your `.env` file as `FOUNDRY_MCP_CONNECTION_ID`

<img src="seq-diagrams/foundry-mcp-connection-id.png" width="80%"/>

> **Note**: The Project connection ID follows this format:
> `/subscriptions/{subscription-id}/resourceGroups/{rg}/providers/Microsoft.CognitiveServices/accounts/{account}/projects/{project}/connections/FoundryMCPServerpreview`

## Architecture Overview
```
┌─────────────────┐     ┌──────────────────┐     ┌─────────────────────────┐
│   User Query    │────▶│   Agent + MCP    │────▶│  Foundry MCP Server     │
│                 │     │     Tool         │     │  (mcp.ai.azure.com)     │
└─────────────────┘     └──────────────────┘     └─────────────────────────┘
                              │                           │
                              ▼                           ▼
                    ┌──────────────────┐       ┌─────────────────────────┐
                    │  Connection ID   │       │  - Model Catalog        │
                    │  (from portal)   │       │  - Deployments          │
                    └──────────────────┘       │  - Evaluations          │
                                               │  - Agent Service        │
                                               └─────────────────────────┘
```

## Available Tool Categories

| Category | Tools | Description |
|----------|-------|-------------|
| **Models** | `model_catalog_list`, `model_deploy` | Explore and deploy models |
| **Evaluation** | `evaluation_create`, `evaluation_get` | Run and review evaluations |
| **Agents** | `agent_update`, `agent_list` | Manage AI agents |
| **Knowledge** | `index_list`, `query_index` | Work with AI Search |

📚 **Learn More**: [Foundry MCP Server Documentation](https://learn.microsoft.com/azure/ai-foundry/mcp/get-started)

## 🔐 Authentication Setup

Before running the next cell, make sure you're authenticated with Azure CLI. Run this command in your terminal:

```bash
az login --use-device-code
```

This will provide you with a device code and URL to authenticate in your browser, which is useful for:
- Remote development environments
- Systems without a default browser
- Corporate environments with strict security policies

After successful authentication, you can proceed with the notebook cells below.

## 1. Initial Setup
We'll load environment variables from `.env` and initialize our **AIProjectClient** to manage agents with MCP tools.

In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv
from azure.identity import AzureCliCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    PromptAgentDefinition,
    MCPTool,
)
from openai.types.responses.response_input_param import (
    McpApprovalResponse,
    ResponseInputParam,
)

# Load environment variables from parent .env
notebook_path = Path().absolute()
parent_dir = notebook_path.parent
load_dotenv(parent_dir.parent / '.env')

# Get configuration
tenant_id = os.environ.get("TENANT_ID")
project_endpoint = os.environ.get("AI_FOUNDRY_PROJECT_ENDPOINT")
model_deployment = os.environ.get("AZURE_AI_MODEL_DEPLOYMENT_NAME", "gpt-4o")

# Foundry MCP Server connection ID (configured in Foundry portal)
foundry_mcp_connection_id = os.environ.get("FOUNDRY_MCP_CONNECTION_ID")

print("🔌 Foundry MCP Server Notebook")
print("=" * 60)
print(f"🔑 Tenant ID: {tenant_id}")
print(f"📍 Project Endpoint: {project_endpoint}")
print(f"🤖 Model Deployment: {model_deployment}")
print("")
print("🔌 Foundry MCP Server: https://mcp.ai.azure.com")
print(f"✅ Connection ID: {foundry_mcp_connection_id}")

## 2. Initialize the AI Project Client
Create the AIProjectClient with Azure CLI-based authentication.

In [ ]:
# Initialize AIProjectClient with Azure CLI authentication
print("🔐 Using Azure CLI authentication...")
print("   Make sure you've run 'az login' in your terminal first!")
print("")

credential = AzureCliCredential(tenant_id=tenant_id)

# Create the project client
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=credential
)

# Get the OpenAI client for conversations
openai_client = project_client.get_openai_client()

print("✅ AIProjectClient initialized with AzureCliCredential")
print("✅ OpenAI client ready for conversations")

## 3. Define the Foundry MCP Tool

The **Foundry MCP Server** is hosted at `https://mcp.ai.azure.com` and provides a curated set of tools for interacting with Microsoft Foundry services.

### Key Parameters:
- **server_label**: Identifies the MCP server (e.g., `"foundry-mcp"`)
- **server_url**: `https://mcp.ai.azure.com/sse` - the cloud-hosted Foundry MCP SSE endpoint
- **require_approval**: Controls approval flow (`"always"`, `"never"`) - use `"never"` for simpler workflows
- **project_connection_id**: **Required** - Connection configured in Foundry portal

### 🔐 Authentication Flow:
1. You configure the MCP connection in Microsoft Foundry portal
2. The portal stores authentication (OAuth/Entra ID)
3. Your code references the connection via `project_connection_id`
4. The agent uses the stored credentials for secure access

### Foundry MCP Server Tools:

| Tool | Description |
|------|-------------|
| `model_catalog_list` | List models from Microsoft Foundry catalog |
| `model_deploy` | Deploy a model to Azure AI Services |
| `model_quota_list` | Check quota availability |
| `evaluation_create` | Create an evaluation run |
| `agent_update` | Create or update an agent |
| `index_list` | List Azure AI Search indexes |

In [ ]:
# Define Foundry MCP Server tool configuration
print("🔌 Configuring Foundry MCP Server tool...")
print("")

# Configure MCP tool with the portal connection
# The server_url should match the "Remote MCP server endpoint" from your connection
mcp_tool = MCPTool(
    server_label="foundry-mcp",
    server_url="https://mcp.ai.azure.com/sse",  # SSE endpoint for MCP protocol
    require_approval="never",  # Skip approval flow for simpler demo
    project_connection_id=foundry_mcp_connection_id,  # Connection for auth
)

print(f"✅ Foundry MCP Tool configured:")
print(f"   Server Label: {mcp_tool.server_label}")
print(f"   Server URL: {mcp_tool.server_url}")
print(f"   Require Approval: {mcp_tool.require_approval}")
print(f"   Connection ID: {mcp_tool.project_connection_id}")

## 4. Create an Agent with MCP Tools

Now we'll create an agent that can use MCP tools to access Foundry capabilities.

In [ ]:
# Define agent instructions for using Foundry MCP Server
agent_name = "foundry-mcp-assistant"

agent_instructions = """
You are an AI Development Assistant with access to Microsoft Foundry capabilities via MCP tools.

Your capabilities include:
1. 📊 Exploring AI models from the Foundry catalog
2. 🚀 Helping with model deployment and quota management
3. 📈 Running evaluations on AI applications
4. 🤖 Managing AI agents in the Foundry ecosystem
5. 🔍 Working with Azure AI Search indexes

When helping users:
- Recommend appropriate models for their use cases (e.g., GPT-4o for complex reasoning, Phi-4 for edge deployments)
- Explain model capabilities, benchmarks, and licensing
- Guide through deployment processes
- Help set up evaluations for quality and safety
- Assist with knowledge base integrations

⚠️ IMPORTANT NOTES:
- Model deployments may require appropriate quota allocations
- Evaluations consume compute resources
- Always verify permissions before making changes
"""

print(f"📝 Agent Name: {agent_name}")
print(f"📋 Instructions length: {len(agent_instructions)} characters")

In [ ]:
# Create the agent with MCP tool
print(f"Creating Foundry MCP Assistant Agent...")

# Create agent with Foundry MCP Server tool
agent_definition = PromptAgentDefinition(
    model=model_deployment,
    instructions=agent_instructions,
    tools=[mcp_tool],
)

# Create the agent version
agent = project_client.agents.create_version(
    agent_name=agent_name,
    definition=agent_definition,
)

print(f"✅ Agent created: {agent.name} (Version: {agent.version})")
print("🔌 Agent configured with Foundry MCP Server tool")
print("")
print("🎯 This agent can now access Foundry MCP Server capabilities:")
print("   • Model exploration and recommendations")
print("   • Deployment management")
print("   • Evaluation orchestration")
print("   • Agent management")

## 5. Create a Conversation

Create a conversation thread to maintain context across interactions.

In [ ]:
# Create a conversation for the chat session
print("Creating conversation for the chat session...")

conversation = openai_client.conversations.create()

print(f"✅ Conversation created: {conversation.id}")

## 6. Query the Agent with Foundry MCP Tools

When the agent uses Foundry MCP tools, you may receive **approval requests**. This is a security feature of the Foundry MCP Server that ensures users approve operations before they execute.

### MCP Approval Flow:
1. Send a query to the agent (e.g., "What models are available?")
2. Agent calls Foundry MCP Server tool (e.g., `model_catalog_list`)
3. Server returns an `mcp_approval_request`
4. Client sends back an `mcp_approval_response` (approve/deny)
5. Agent receives data and responds to user

### Example Prompts for Foundry MCP Server:
- "What OpenAI models are available in Microsoft Foundry?"
- "Compare Phi models and explain their differences"
- "What models are good for reasoning tasks?"
- "Help me understand model quotas for my subscription"

In [ ]:
def process_mcp_response(openai_client, agent, initial_response, auto_approve=True):
    """
    Process a response that may contain MCP approval requests.
    
    Args:
        openai_client: The OpenAI client instance
        agent: The agent being used
        initial_response: The initial response from the agent
        auto_approve: Whether to automatically approve MCP requests
    
    Returns:
        The final response after processing any approvals
    """
    response = initial_response
    
    # Check for MCP approval requests
    approval_requests = []
    for item in response.output:
        if item.type == "mcp_approval_request":
            print(f"\n🔐 MCP Approval Request Received:")
            print(f"   Server: {item.server_label}")
            print(f"   Request ID: {item.id}")
            
            if auto_approve and item.id:
                print(f"   ✅ Auto-approving request...")
                approval_requests.append(
                    McpApprovalResponse(
                        type="mcp_approval_response",
                        approve=True,
                        approval_request_id=item.id,
                    )
                )
            else:
                print(f"   ❌ Request denied or missing ID")
    
    # If we have approvals to send, send them
    if approval_requests:
        print(f"\n📤 Sending {len(approval_requests)} approval response(s)...")
        
        response = openai_client.responses.create(
            input=approval_requests,
            previous_response_id=response.id,
            extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
        )
        
        print("✅ Approval response sent")
    
    return response

print("✅ MCP response processing function defined")

In [ ]:
# Sample queries that leverage Foundry MCP Server capabilities
queries = [
    "What AI models would you recommend for document analysis use cases?",
    "Explain how the Foundry MCP Server helps with AI development workflows.",
    "What security features does the Foundry MCP Server provide for enterprise use?",
]

print("💬 Foundry MCP Server Assistant Session")
print("=" * 60)

for i, query in enumerate(queries, 1):
    print(f"\n📝 Query {i}: {query}")
    print("-" * 50)
    
    # Send the query
    response = openai_client.responses.create(
        conversation=conversation.id,
        input=query,
        extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
    )
    
    # Process any MCP approval requests
    response = process_mcp_response(openai_client, agent, response)
    
    # Display the response
    print(f"\n🤖 Agent Response:")
    print(response.output_text if response.output_text else "No response text")
    print("\n" + "=" * 60)

## 7. Interactive MCP Query (Optional)

Try your own queries with the MCP-enabled agent.

In [ ]:
# Interactive query - try Foundry MCP Server prompts
# Example prompts you can try:
# - "What are the most popular models in Microsoft Foundry?"
# - "Show me models with MIT license"
# - "What models from Meta are available?"
# - "Compare GPT-4o and Phi-4 for my use case"

custom_query = "What capabilities does the Foundry MCP Server provide for building AI agents?"

print(f"📝 Custom Query: {custom_query}")
print("-" * 50)

# Send the query
response = openai_client.responses.create(
    conversation=conversation.id,
    input=custom_query,
    extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
)

# Process any MCP approval requests
response = process_mcp_response(openai_client, agent, response)

# Display the response
print(f"\n🤖 Agent Response:")
print(response.output_text if response.output_text else "No response text")

## 8. Cleanup

Delete the agent when done to clean up resources.

In [ ]:
# Cleanup: Delete the agent
print("🧹 Cleaning up resources...")

try:
    project_client.agents.delete_version(
        agent_name=agent.name,
        agent_version=agent.version
    )
    print(f"✅ Agent deleted: {agent.name} (Version: {agent.version})")
except Exception as e:
    print(f"⚠️ Error during cleanup: {e}")

print("\n🎉 MCP Tools notebook completed!")

## 📚 Summary

In this notebook, we learned how to use the **Foundry MCP Server** to enhance AI agents with Foundry capabilities.

### Key Setup Steps:

1. **Configure in Portal**: Create MCP connection in Microsoft Foundry portal
2. **Get Connection ID**: Copy the connection ID from the portal
3. **Add to .env**: Set `FOUNDRY_MCP_CONNECTION_ID` in your environment
4. **Create Agent**: Use `MCPTool` with `project_connection_id`

### Key Concepts:

| Concept | Description |
|---------|-------------|
| **Foundry MCP Server** | Cloud-hosted MCP service at `https://mcp.ai.azure.com` |
| **MCPTool** | Tool configuration for connecting to the server |
| **project_connection_id** | **Required** - Connection configured in Foundry portal |
| **Entra ID Auth** | Secure authentication with user-specific permissions |
| **Approval Flow** | Security feature requiring user consent |

### Foundry MCP Server Tool Categories:

| Category | Example Tools | Use Cases |
|----------|--------------|-----------|
| **Models** | `model_catalog_list`, `model_deploy`, `model_quota_list` | Explore, compare, and deploy AI models |
| **Evaluations** | `evaluation_create`, `evaluation_get` | Quality and safety assessments |
| **Agents** | `agent_update`, `agent_list` | Agent lifecycle management |
| **Knowledge** | `index_list`, `query_index` | Azure AI Search integration |

### Example Prompts for Foundry MCP Server:

```
# Model Exploration
"What OpenAI models are available in Microsoft Foundry?"
"Compare Phi models and explain their differences"
"What models are good for reasoning tasks?"

# Deployment
"What quota do I have available for GPT-4o?"
"Help me deploy a model for my project"

# Evaluation
"How do I evaluate my agent's quality?"
"What evaluators are available for text generation?"

# General Use Cases
"Recommend models for document analysis"
"What models have appropriate licensing for enterprise use?"
```

### 🔐 Security Best Practices:

- ✅ Foundry MCP Server uses **Entra ID authentication**
- ✅ All operations run under **user's Azure RBAC permissions**
- ✅ Every activity is **logged for auditing**
- ✅ Use `require_approval="never"` for simpler demos, or `"always"` for production with manual approval
- ✅ Tenant admins can enforce access via **Azure Policy**

### 🚀 Using Foundry MCP Server in VS Code

You can also use Foundry MCP Server directly in VS Code with GitHub Copilot:

1. Open VS Code with GitHub Copilot enabled
2. Add to `.vscode/mcp.json`:
```json
{
  "servers": {
    "foundry-mcp-remote": {
      "type": "http",
      "url": "https://mcp.ai.azure.com"
    }
  }
}
```
3. Start chatting with Copilot using Foundry tools!

### 📚 Resources:

- [Foundry MCP Server Documentation](https://learn.microsoft.com/azure/ai-foundry/mcp/get-started)
- [Available Tools Reference](https://learn.microsoft.com/azure/ai-foundry/mcp/available-tools)
- [Security Best Practices](https://learn.microsoft.com/azure/ai-foundry/mcp/security-best-practices)
- [Microsoft Foundry Blog - MCP Announcement](https://devblogs.microsoft.com/foundry/announcing-foundry-mcp-server-preview-speeding-up-ai-dev-with-microsoft-foundry/)
- [GitHub: mcp-foundry (legacy reference)](https://github.com/microsoft-foundry/mcp-foundry)